<a href="https://colab.research.google.com/github/zigzager/SF/blob/main/generador_intervalo_v3.23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- INICIO DE CELDA 1 ---
%%writefile visualizador_core.py
# --- ARCHIVO 1: visualizador_core.py ---
import pandas as pd
import matplotlib.pyplot as plt

def crear_grafico_interno(ruta_csv, tipo_grafico, fecha_inicio, fecha_fin, nombre_archivo_salida, niveles_destacados=[]):

    # 1. Carga
    try:
        df = pd.read_csv(ruta_csv, sep='\t', parse_dates={'Datetime': ['<DATE>', '<TIME>']})
        df.rename(columns={'<OPEN>': 'Open', '<CLOSE>': 'Close', '<HIGH>': 'High', '<LOW>': 'Low'}, inplace=True)
        df.set_index('Datetime', inplace=True)
        df = df[['Open', 'High', 'Low', 'Close']]
    except Exception as e:
        print(f"Error carga: {e}")
        return

    # 2. Filtro
    datos = df.loc[fecha_inicio : fecha_fin].copy()
    if datos.empty:
        print("Error: Sin datos en ese rango.")
        return

    # 3. Lienzo (Tamaño Estándar de Monitor)
    # Usamos 12x6 pulgadas. Es un tamaño panorámico estándar.
    # Al ser fijo, si metes muchas barras, se apretarán solas.
    fig, ax = plt.subplots(figsize=(12, 6), dpi=300)

    # --- ESTILO: FONDO BLANCO ---
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    # Opcional: Una rejilla muy suave como NinjaTrader (ayuda a ver la escala)
    ax.grid(True, color='gray', linestyle='-', linewidth=0.3, alpha=0.3)

    # --- CONFIGURACIÓN FIJA (MÉTRICAS NINJATRADER) ---
    lw = 1.1  # Grosor de línea fijo
    ms = 2.0  # Tamaño de punto fijo
    col = 'black' # Color único para todo
    # -------------------------------------------------

    # 4. Dibujo
    x = range(len(datos))
    y_c, y_l, y_h = datos['Close'].values, datos['Low'].values, datos['High'].values

    # Dibujamos todo de una pasada (más rápido y limpio)
    ax.vlines(x, ymin=y_l, ymax=y_h, color=col, linewidth=lw)
    ax.plot(x, y_c, 'o', color=col, markersize=ms)

    # 5. Niveles (Soportes)
    if niveles_destacados:
        print(f"   -> Pintando {len(niveles_destacados)} zonas.")
        for nivel in niveles_destacados:
            # Línea de nivel: Dorada oscura para que se vea en blanco
            ax.axhline(y=nivel, color='#d4af37', linewidth=1.5, alpha=0.6, linestyle='-')

    # 6. Guardar
    ax.axis('off')

    # Ajustamos los márgenes para aprovechar toda la imagen
    ax.set_xlim(-0.5, len(datos) - 0.5)

    fig.subplots_adjust(left=0.01, right=0.99, top=0.99, bottom=0.01)
    plt.savefig(nombre_archivo_salida, format='png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    print(f"\n¡Imagen generada! (Estilo NinjaTrader | Velas: {len(datos)})")
# --- FIN DE CELDA 1 ---

Writing visualizador_core.py


In [2]:
# --- INICIO DE CELDA 2 ---
%%writefile scanner.py
# --- ARCHIVO 3: scanner.py ---
import pandas as pd
from datetime import timedelta
from scipy.signal import find_peaks

# --- Apartado S-2: Algoritmo Puro ---
def encontrar_niveles(df_zona, sensibilidad, prominencia, margen):
    # Fase 1: Pivotes
    picos, _ = find_peaks(df_zona['High'], distance=sensibilidad, prominence=prominencia)
    valles, _ = find_peaks(-df_zona['Low'], distance=sensibilidad, prominence=prominencia)

    todos_pivotes = pd.concat([df_zona.iloc[picos]['High'], df_zona.iloc[valles]['Low']])

    if todos_pivotes.empty: return []

    # Fase 2: Agrupación (Zonas)
    pivotes_ordenados = todos_pivotes.sort_values()
    zonas = []

    while not pivotes_ordenados.empty:
        base = pivotes_ordenados.iloc[0]
        # Buscamos pivotes cercanos (dentro del margen)
        en_zona = pivotes_ordenados[pivotes_ordenados <= base + margen]

        if not en_zona.empty:
            zonas.append((en_zona.mean(), len(en_zona))) # (PrecioMedio, Hits)
            pivotes_ordenados = pivotes_ordenados[pivotes_ordenados > base + margen]
        else:
            pivotes_ordenados = pivotes_ordenados.iloc[1:]

    return zonas

# --- Apartado S-3: Función Maestra ---
def obtener_niveles_rango(ruta_csv, fecha_inicio, fecha_fin, config):
    """
    Carga el CSV y devuelve la lista de precios clave.
    'config' es un diccionario con: sensibilidad, margen, prominencia, num_niveles.
    """
    try:
        df = pd.read_csv(ruta_csv, sep='\t', parse_dates={'Datetime': ['<DATE>', '<TIME>']})
        df.rename(columns={'<HIGH>': 'High', '<LOW>': 'Low'}, inplace=True)
        df.set_index('Datetime', inplace=True)

        # +120 mins de margen para asegurar que el scanner ve el contexto final
        df_zona = df.loc[fecha_inicio : fecha_fin + timedelta(minutes=120)].copy()

        if df_zona.empty: return []

        # Ejecutar cerebro
        zonas = encontrar_niveles(df_zona, config['N'], config['PROM'], config['MARGEN'])

        # Ordenar por Hits y devolver los Top N
        zonas.sort(key=lambda x: x[1], reverse=True)
        mejores = zonas[:config['TOP']]

        return [z[0] for z in mejores]

    except Exception as e:
        print(f"Error Scanner: {e}")
        return []
# --- FIN DE CELDA 2 ---

Writing scanner.py


In [3]:
# --- INICIO DE CELDA 3 (APP PRINCIPAL) ---
from datetime import datetime
import uuid
# Importamos nuestras herramientas
from visualizador_core import crear_grafico_interno
from scanner import obtener_niveles_rango

# --- CONFIGURACIÓN ---
PARAMS_M1 = {'N': 20, 'MARGEN': 1.5, 'PROM': 1.5, 'TOP': 2}
PARAMS_H1 = {'N': 5,  'MARGEN': 3.0, 'PROM': 2.0, 'TOP': 2}

# RUTAS
FILE_M1 = "M1_11_2025.csv"
FILE_H1 = "XAUUSD_H1_200406110700_202511142300.csv"

def ejecutar_app():
    while True:
        print("\n" + "="*40)
        print("   🎯 MENÚ PRINCIPAL (V3.4 FIXED)")
        print("="*40)
        print("1. Gráfico 1 MINUTO (M1)")
        print("2. Gráfico 1 HORA   (H1)")
        print("q. Salir")

        opcion = input("\n👉 Elige opción: ")

        if opcion.lower() == 'q': break

        if opcion == '1':
            archivo, cfg, fmt, tipo, txt = FILE_M1, PARAMS_M1, '%Y-%m-%d %H:%M', '1', "M1"
        elif opcion == '2':
            archivo, cfg, fmt, tipo, txt = FILE_H1, PARAMS_H1, '%Y-%m-%d %H', '2', "H1"
        else:
            print("Opción no válida.")
            continue

        # Pedir Fechas
        print(f"\n--- Fechas para {txt} (Ej: {datetime.now().strftime(fmt)}) ---")
        try:
            ini_str = input("Inicio: ")
            inicio = datetime.strptime(ini_str, fmt)
            fin_str = input("Fin:    ")
            fin = datetime.strptime(fin_str, fmt)
            if fin <= inicio:
                print("❌ Error: Fin menor que Inicio.")
                continue
        except ValueError:
            print("❌ Error formato fecha.")
            continue

        # 1. SCANNER
        print(f"\n🔎 Buscando niveles...")
        niveles = obtener_niveles_rango(archivo, inicio, fin, cfg)

        if niveles:
            print(f"✅ ¡Encontrados!: {[round(n, 2) for n in niveles]}")
        else:
            print("⚠️ No se encontraron niveles claros.")

        # 2. DIBUJAR (Con nombre ÚNICO)
        # Aquí he vuelto a añadir el uuid.hex[:6] para que no se repitan nombres
        codigo_unico = uuid.uuid4().hex[:6]
        nombre_img = f"ANALISIS_{txt}_{inicio.strftime('%d%H%M')}_{fin.strftime('%d%H%M')}_{codigo_unico}.png"

        crear_grafico_interno(archivo, tipo, inicio, fin, nombre_img, niveles)

if __name__ == "__main__":
    ejecutar_app()
# --- FIN DE CELDA 3 ---


   🎯 MENÚ PRINCIPAL (V3.4 FIXED)
1. Gráfico 1 MINUTO (M1)
2. Gráfico 1 HORA   (H1)
q. Salir

👉 Elige opción: 2

--- Fechas para H1 (Ej: 2025-11-17 22) ---
Inicio: 2025-10-10 19
Fin:    2025-11-10 19

🔎 Buscando niveles...


/content/scanner.py:40: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(ruta_csv, sep='\t', parse_dates={'Datetime': ['<DATE>', '<TIME>']})


✅ ¡Encontrados!: [np.float64(3965.23), np.float64(3971.76)]


/content/visualizador_core.py:9: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(ruta_csv, sep='\t', parse_dates={'Datetime': ['<DATE>', '<TIME>']})


   -> Pintando 2 zonas.

¡Imagen generada! (Estilo NinjaTrader | Velas: 484)

   🎯 MENÚ PRINCIPAL (V3.4 FIXED)
1. Gráfico 1 MINUTO (M1)
2. Gráfico 1 HORA   (H1)
q. Salir

👉 Elige opción: 1

--- Fechas para M1 (Ej: 2025-11-17 22:31) ---
Inicio: 2025-11-17 14:10
Fin:    2025-11-17 21:07

🔎 Buscando niveles...
✅ ¡Encontrados!: [np.float64(4071.58), np.float64(4074.43)]
   -> Pintando 2 zonas.


/content/scanner.py:40: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(ruta_csv, sep='\t', parse_dates={'Datetime': ['<DATE>', '<TIME>']})
/content/visualizador_core.py:9: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(ruta_csv, sep='\t', parse_dates={'Datetime': ['<DATE>', '<TIME>']})



¡Imagen generada! (Estilo NinjaTrader | Velas: 418)

   🎯 MENÚ PRINCIPAL (V3.4 FIXED)
1. Gráfico 1 MINUTO (M1)
2. Gráfico 1 HORA   (H1)
q. Salir


KeyboardInterrupt: Interrupted by user